In [40]:
import sys

sys.path.append("../src/stong_core_cython")
sys.path.append("../src/stong_core_cython/md")

In [41]:
import numpy as np
import pandas as pd
import hartree_fock
import basis_data
import importlib
import matrix
import two_electron
from scipy.optimize import minimize

importlib.reload(hartree_fock)
importlib.reload(basis_data)
importlib.reload(matrix)
importlib.reload(two_electron)

from atom import *
from hartree_fock import *
from basis_data import *
from matrix import *
from two_electron import *

In [42]:
df = pd.read_csv("../examples/make_base/sto_3g.csv")

In [43]:
zetas = [1, 1, 1]
basis_data = {"H": create_basis_data(df, zetas, 3, 1)}
molecule = Molecule(
    [Atom("H", np.array([0.0, 0.0, 0.0])), Atom("H", np.array([0.0, 0.0, 0.74]))]
)
basis_functions = create_basis_function(molecule, basis_data)